# Worm-eating Warbler
This notebook is a tool for exploring data sets requested from GBIF (and eventually other sources), and mostly for developing criteria for filtering records (filter sets).  When the entire notebook is run, it retrieves records according to the filter sets specified and saves the results (records and some summary tables) in an sqlite database.  Some information is pulled from the parameters.sqlite database that is saved in this repository.

### General Setup

In [1]:
%matplotlib inline
import sqlite3
import pprint
import json
import pandas as pd
import numpy as np
#import geopandas as gpd
pd.set_option('display.width', 600)
pd.set_option('display.max_colwidth', 60)
pd.set_option('display.max_rows', 150)
from IPython.display import Image
from pygbif import occurrences
import matplotlib.pyplot as plt
import os
from datetime import datetime
t1 = datetime.now()
%cd ..
import repo_functions as functions

T:\code\occurrence-records-wrangler


There is a bug with mpl_toolkits, the following code is a temp fix, hopefully.
https://stackoverflow.com/questions/52911232/basemap-library-using-anaconda-jupyter-notebooks-keyerror-proj-lib/54087410#54087410

In [2]:
os.environ['PROJ_LIB'] = r'c:\Users\nmtarr\AppData\Local\Continuum\miniconda3\envs\occurrences\Library\share'

### Set Variables
Fill out stuff in this cell.

In [3]:
species_id = 'bwewax0'
summary_name = 'wormieSPD'
gbif_req_id = 'GBIFr9'
gbif_filter_id = 'GBIFf9'
workDir = 'T:/Occurrence_Records/'
codeDir = 'T:/Code/occurrence-records-wrangler/'
inDir = workDir + 'Inputs/'
outDir = workDir + 'Outputs/'
default_coordUncertainty = 200
SRID_dict = {'WGS84': 4326, 'AlbersNAD83': 102008} # Used in file names for output.
spdb = outDir + "Speed/" + species_id + gbif_req_id + gbif_filter_id + '.sqlite'

print("Notebook run " + str(t1))

Notebook run 2019-12-02 11:34:23.050166


In [4]:
connjup = sqlite3.connect(codeDir + 'parameters.sqlite')
cursorjup = connjup.cursor()

# Get some variables
years = connjup.execute("""SELECT years_range 
                           FROM gbif_requests WHERE request_id = '{0}'""".format(gbif_req_id)).fetchone()[0]
gap_id = connjup.execute("""SELECT gap_id
                            FROM species_concepts WHERE species_id = '{0}'""".format(species_id)).fetchone()[0]
common_name = connjup.execute("""SELECT common_name
                                 FROM species_concepts WHERE species_id = '{0}'""".format(species_id)).fetchone()[0]

# Species Concept
Display information on the species from the parameters.sqlite database.

In [5]:
vals = cursorjup.execute("SELECT * FROM species_concepts WHERE species_id = '{0}';".format(species_id)).fetchall()[0]
cols = [x[1] for x in cursorjup.execute("PRAGMA table_info('species_concepts')").fetchall()]
pprint.pprint(dict(zip(cols, vals)))

{'bcb_id': None,
 'breeding_months': '5,6,7',
 'common_name': 'worm-eating warbler',
 'concept_vetted': 0,
 'detection_distance_meters': 200,
 'ebird_id': None,
 'end_year': None,
 'error_tolerance': 20,
 'fws_id': '',
 'gap_id': 'bwewax',
 'gbif_id': '2489603',
 'geometry': None,
 'itis_tsn': '726195',
 'migratory': '1',
 'notes': 'Species name changed from vermivorus in 2003.  Breeding and '
          'wintering months from Birds of North America Database (July 26, '
          '2019).  Migration is possible in early May and late July. N. Tarr '
          '26 July 2019.',
 'pad': 1,
 'scientific_name': 'Helmitheros vermivorum',
 'species_id': 'bwewax0',
 'start_year': 2003,
 'wintering_months': '10,11,12,1,2'}


# Filters
Display the parameters of the request filter set.  These are deployed during the step where records are retrieved from the API.

In [6]:
df1 = pd.read_sql_query(sql="SELECT * FROM gbif_requests WHERE request_id = '{0}'".format(gbif_req_id), con=connjup)
print("THE REQUEST FILTER SET")
print(df1.loc[0])

THE REQUEST FILTER SET
request_id                     GBIFr9
source                           GBIF
lat_range                       27,41
lon_range                     -91,-75
years_range                 2000,2020
months_range                     1,12
geoissue                         None
coordinate                       True
country                          None
geometry                         None
continent                        None
creator                       N. Tarr
notes           Used for speed tests.
Name: 0, dtype: object


Display the parameters of the post-request filter set.  These are deployed after the records are retrieved from the API, but before they are stored in the occurrence record sqlite database.

In [7]:
df2 = pd.read_sql_query(sql="SELECT * FROM gbif_filters WHERE filter_id = '{0}'".format(gbif_filter_id), con=connjup)
print("THE POST REQUEST FILTER SET")
print(df2.loc[0])

THE POST REQUEST FILTER SET
filter_id                                                               GBIFf9
dataset                                                                   GBIF
collection_codes_omit                                                         
institutions_omit                                                             
has_coordinate_uncertainty                                                   0
max_coordinate_uncertainty                                               10000
bases_omit                                                                    
protocols_omit                                                                
sampling_protocols_omit                                                       
issues_omit                                                                   
duplicates_OK                                                             True
creator                                                                N. Tarr
notes                   

In [8]:
functions.retrieve_gbif_occurrences(codeDir, species_id, inDir, spdb, gbif_req_id, gbif_filter_id, 
                                    default_coordUncertainty, SRID_dict, outDir, summary_name)

SPATIALITE_SECURITY set to relaxed
Created occurrence db: 0:00:06.602384
Got request params and sorted out geometry constraints: 0:00:00.002975


KeyboardInterrupt: 

### How many records made it through the filters?
This is the number that was actually saved in the occurrence record sqlite database.

In [ ]:
conn_occ= sqlite3.connect(spdb)
curs_occ = conn_occ.cursor()
record_count = curs_occ.execute("SELECT COUNT(occ_id) FROM occurrences WHERE species_id = '{0}'".format(species_id)).fetchone()
print(str(record_count[0]) + " records")

# Descriptions of filtered records

### Locations

In [ ]:
shp1 = {'file': '{0}{1}_circles'.format(outDir, summary_name), 'column': None,
        'alias': 'Occurrence records', 'drawbounds': True, 'linewidth': .75, 'linecolor': 'k',
        'fillcolor': None, 'marker':'o'}

# Display occurrence polygons
map_these=[shp1]
    
title="{1} ({0})".format(years, common_name)
functions.MapShapefilePolygons(map_these=map_these, title=title)

### Runtime

In [ ]:
t2 = datetime.now()
print(t2 - t1)